In [ ]:
#from ../../Setup/secret_wifi/config.py import *
# for privat channels
import sys
sys.path.insert(0, '../../Setup/setup')
from config_thingspeak import config

In [ ]:
import pandas as pd
import numpy as np
import requests
import json
import plotly.express as px
import matplotlib.pyplot as plt

In [ ]:
READ_API_KEY = config['user_api_key']
CHANNEL_ID = '1540609'
NUMBER_ENTRIES = '300'  # amount of data points returned

In [ ]:
# get channel ID
req_channel_id = requests.get('https://api.thingspeak.com/channels.json?api_key='+ READ_API_KEY)
req_channel_id.text[0:13]

# [Read data from ThingSpeak](https://ch.mathworks.com/help/thingspeak/readdata.html)

In [ ]:
# get data string, result = number of entrys (max. 8000)
read_data = 'https://api.thingspeak.com/channels/' + CHANNEL_ID + '/feeds.json?results=' + NUMBER_ENTRIES
read_data

In [ ]:
# get request json data
req = requests.get(read_data)
#req_json.text

In [ ]:
# set respond text to json
json_data = json.loads(req.text)
json_data

In [ ]:
# get respond keys
json_data.keys()

In [ ]:
# get data to panda df
df_data = pd.json_normalize(json_data['feeds'])
df_data.head()
#df_data.dtypes

In [ ]:
# set columns names
df_data = df_data.rename(columns={'created_at': 'timestamp',
                                'entry_id': 'id',
                                'field1': 'Temperature',
                                'field2': 'Humiditiy',
                                'field3': 'Light',
                                'field4': 'Weight',
                                'field5': 'Th_Weight',
                                'field6': 'need water'})
# set datetime
df_data['timestamp'] = pd.to_datetime(df_data['timestamp'])
# set dtypes
numeric_cols = ['Temperature', 'Humiditiy', 'Light', 'Weight', 'Th_Weight', 'need water']
df_data[numeric_cols] = df_data[numeric_cols].astype(float).astype(int)  # solve 0.0 values with astype(float) first
# set date as index
df_data = df_data.set_index('timestamp', drop=True)
print(df_data.shape)
df_data.head(3)

In [ ]:
# get data with file reader
# print(type(req_json.text))
# with open(req_json.text) as json_data:  #, 'r', encoding='UTF-8'
#     req_data = json.load(json_data)
#     print(req_data)

# Clean Data

In [ ]:
(df_data == 0).sum()

In [ ]:
# replace 0 with nan
cols = ['Temperature', 'Humiditiy', 'Light', 'Weight', 'Th_Weight']
df_data[cols] = df_data[cols].replace(0, np.nan)
#print(df_data.isna().sum())
df_data = df_data.dropna()
print(df_data.isna().sum())

# Visualization Plotly Express

In [ ]:
fig = px.line(df_data[['Temperature', 'Humiditiy']])
fig.update_layout(title='Thingspeak Data')
fig.show()

In [ ]:
fig = px.line(df_data[['Weight', 'Th_Weight', 'need water']])
fig.update_layout(title='Thingspeak Data')
fig.show()

# Visalization Matplotlib

In [ ]:
plt.plot(df_data[['Temperature', 'Humiditiy']])
plt.show()